# 修正index

In [ ]:
from src import align

align.correct_index(root_dir="/home/ljw/sdb1/naapam")

# 统计score分布

In [ ]:
from src import main

main.stat_read(root_dir="/home/ljw/sdb1/naapam")

# 收集align结果

In [ ]:
from src import main

df_alg = main.collect_data(root_dir="/home/ljw/sdb1/naapam", min_score=-1)
df_alg.to_feather("/home/ljw/sdb1/naapam/alg.feather")

# 统计mutant指标

In [ ]:
from src import main

main.stat_mutant(df_alg)

# 按mutant筛选

In [ ]:
import yaml
from src import main

with open("filter_configs/mutant.yaml", "r") as fd:
    params = yaml.load(fd, Loader=yaml.CLoader)

df_alg = main.filter_mutant(df_alg, **params)

# 统计ref指标

In [ ]:
from src import main

main.stat_ref(df_alg)

# 筛选ref指标

In [ ]:
import yaml
from src import main

with open("filter_configs/ref.yaml", "r") as fd:
    params = yaml.load(fd, Loader=yaml.CLoader)

df_alg = main.filter_ref(df_alg, **params)

# 查看1bp删除造成的dummy模板插入的频率

In [ ]:
from src import utils, draw

for tem in range(1,5):
    draw.over_up_del_size_on_tem(df_alg.assign(
        freq_without_wt=lambda df: df["count"] / (df.groupby(["stem", "ref_id"])["count"].transform("sum") - utils.count_wt(df_alg) + 1e-6)
    ), tem, ["freq_without_wt"], aggfunc="sum")